# 1. Feature Engineering

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from sklearn.model_selection import KFold, train_test_split

#builted package
from rwc19 import dataManager, MultiOutputRF, norme2, get_loss_win_accuracy, fit_predict_new

In [3]:
#Chargement des données
data = pd.read_csv('Dataset/Games_2015_2019_with_rankings.csv', sep=';')
data = data.iloc[:,1:]
data.head()

,Team,Result,For,Aga,Diff,HTf,HTa,Opposition,Ground,Match Date,TeamRk,OppRk
0,wales,lost,16.0,21.0,-5.0,16.0,8.0,england,Millennium Stadium,06/02/2015,81.64,84.85
1,france,won,15.0,8.0,7.0,9.0,8.0,scotland,Stade de France,07/02/2015,79.66,78.78
2,scotland,lost,8.0,15.0,-7.0,8.0,9.0,france,Stade de France,07/02/2015,78.78,79.66
3,italy,lost,3.0,26.0,-23.0,3.0,9.0,ireland,Rome,07/02/2015,71.19,85.48
4,ireland,won,26.0,3.0,23.0,9.0,3.0,italy,Rome,07/02/2015,85.48,71.19


In [4]:
#Quelque sinformations sur la base
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 761 entries, 0 to 760
Data columns (total 12 columns):
Team          761 non-null object
Result        741 non-null object
For           741 non-null float64
Aga           741 non-null float64
Diff          741 non-null float64
HTf           741 non-null float64
HTa           741 non-null float64
Opposition    761 non-null object
Ground        761 non-null object
Match Date    761 non-null object
TeamRk        729 non-null float64
OppRk         733 non-null float64
dtypes: float64(7), object(5)
memory usage: 71.4+ KB


In [5]:
# Afficher les lignes qui contiennent les valeurs manquantes
data.loc[data.isna().sum(axis=1)>0,:]

,Team,Result,For,Aga,Diff,HTf,HTa,Opposition,Ground,Match Date,TeamRk,OppRk
729,russia,lost,9.0,34.0,-25.0,6.0,5.0,samoa,Japan,24/09/2019,NaN,67.232921
730,fiji,lost,27.0,30.0,-3.0,12.0,24.0,uruguay,Japan,25/09/2019,NaN,66.188227
731,italy,won,48.0,7.0,41.0,17.0,0.0,canada,Japan,26/09/2019,NaN,56.723461
732,england,won,45.0,7.0,38.0,19.0,0.0,united states of america,Japan,26/09/2019,NaN,72.493840
733,argentina,won,28.0,12.0,16.0,28.0,7.0,tonga,Japan,28/09/2019,NaN,NaN
734,japan,won,19.0,12.0,7.0,9.0,12.0,ireland,Japan,28/09/2019,NaN,NaN
735,south africa,won,57.0,3.0,54.0,31.0,3.0,namibia,Japan,28/09/2019,NaN,NaN
736,georgia,won,33.0,7.0,26.0,12.0,7.0,uruguay,Japan,29/09/2019,NaN,NaN
737,australia,lost,25.0,29.0,-4.0,8.0,23.0,wales,Japan,29/09/2019,NaN,NaN
738,scotland,won,34.0,0.0,34.0,20.0,0.0,samoa,Japan,30/09/2019,NaN,NaN


Le ranking de la team à domicile est absente pour toutes ces observations recensées, il y a également des matchs nons joués encore au moment du datachallenge. Nous les supprimons simplement de la base. 

In [6]:
data.dropna().shape

(729, 12)

Les traitements se feront donc sur 729 matchs au total.

In [7]:
dM = dataManager(dataframe=data.dropna())

In [8]:
dM.data.sample(5)

,Team,Result,For,Aga,Diff,HTf,HTa,Opposition,Ground,Match Date,TeamRk,OppRk
4,ireland,won,26.0,3.0,23.0,9.0,3.0,italy,Rome,07/02/2015,85.480000,71.190000
634,france,lost,8.0,44.0,-36.0,8.0,30.0,england,Twickenham,10/02/2019,78.348817,88.372516
500,france,lost,13.0,14.0,-1.0,10.0,14.0,wales,Millennium Stadium,17/03/2018,79.123758,85.483126
358,argentina,lost,34.0,38.0,-4.0,17.0,13.0,england,San Juan,10/06/2017,77.381347,90.376841
474,france,lost,13.0,15.0,-2.0,3.0,9.0,ireland,Stade de France,03/02/2018,76.621152,87.791640


In [9]:
# Ajout des différentes feautures pour la colonne date
dM.generate_date_features('Match Date')

'4 features successfully generated'

In [10]:
dM.data.sample(5)

,Team,Result,For,Aga,Diff,HTf,HTa,Opposition,Ground,Match Date,TeamRk,OppRk,year,month,day0fYear,dayOfWeek
659,south africa,won,35.0,17.0,18.0,14.0,10.0,australia,Johannesburg,2019-07-20,85.152969,79.870069,2019,7,201,5
316,ireland,won,27.0,24.0,3.0,17.0,7.0,australia,Lansdowne Road,2016-11-26,86.729367,86.167762,2016,11,331,5
246,united states of america,won,25.0,0.0,25.0,15.0,0.0,russia,Sacramento,2016-06-25,67.258516,57.833002,2016,6,177,5
477,canada,lost,31.0,32.0,-1.0,18.0,10.0,uruguay,Montevideo,2018-03-02,59.585224,66.358322,2018,3,61,4
308,france,lost,23.0,25.0,-2.0,11.0,13.0,australia,Stade de France,2016-11-19,80.783447,88.822311,2016,11,324,5


# 2. Prédiction de quelques matchs pour tester l'algorithme

In [11]:
#On enlève les colonnes qui ne servent pas pour l'entrainement du modèle
cols_to_drop = ['Match Date', 'Result', 'Diff', 'HTf', 'HTa', 'Ground']
try:
    dM.data.drop(cols_to_drop, axis=1, inplace=True)
except:
    print("Les noms d'une ou plusieurs colonnes sont mal spécifiées")
dM.data.head()

,Team,For,Aga,Opposition,TeamRk,OppRk,year,month,day0fYear,dayOfWeek
0,wales,16.0,21.0,england,81.64,84.85,2015,6,153,1
1,france,15.0,8.0,scotland,79.66,78.78,2015,7,183,3
2,scotland,8.0,15.0,france,78.78,79.66,2015,7,183,3
3,italy,3.0,26.0,ireland,71.19,85.48,2015,7,183,3
4,ireland,26.0,3.0,italy,85.48,71.19,2015,7,183,3


In [12]:
# On applique du one-hot-encoding sur toutes les variables catégorielles.
full_X = pd.get_dummies(dM.data, drop_first=False) #tous les k catégories sont pris en compte
full_X.head()

,For,Aga,TeamRk,OppRk,year,month,day0fYear,dayOfWeek,Team_argentina,Team_australia,...,Opposition_namibia,Opposition_new zealand,Opposition_russia,Opposition_samoa,Opposition_scotland,Opposition_south africa,Opposition_tonga,Opposition_united states of america,Opposition_uruguay,Opposition_wales
0,16.0,21.0,81.64,84.85,2015,6,153,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,15.0,8.0,79.66,78.78,2015,7,183,3,0,0,...,0,0,0,0,1,0,0,0,0,0
2,8.0,15.0,78.78,79.66,2015,7,183,3,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3.0,26.0,71.19,85.48,2015,7,183,3,0,0,...,0,0,0,0,0,0,0,0,0,0
4,26.0,3.0,85.48,71.19,2015,7,183,3,0,0,...,0,0,0,0,0,0,0,0,0,0


Il faut noter que le one-hot-encoding des variables catégorielles n'est là que pour spécifier quelle équipe joue contre quelle équipe, on ne s'attend donc pas à ces variables aient une quelconque importance en terme de pouvoir prédictif dans le modèle.

In [13]:
# Dissociation de la matrice des régresseurs X et des vecteurs à prédire y
y_cols = ['For', 'Aga']
X, y = full_X.drop(y_cols, axis=1), full_X[y_cols]

In [14]:
# Découpage en train/test de la base
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# Entrainement et prédictions avec 2000 arbres
clf = MultiOutputRF(n_estimators=2000).fit(X_train, y_train)
Ypred = clf.predict(X_test)

In [16]:
# Ce que l'algorithme prédit comme score des 5 derniers matchs dans la base test
y_pred_df = pd.DataFrame(Ypred.astype(int), columns=["For", 'Aga'])
y_pred_df.tail()

,For,Aga
141,29,18
142,44,19
143,23,25
144,27,20
145,25,20


In [17]:
# Et voici les vrais scores des 5 derniers matchs dans la base test
y_test.astype(int).tail()

,For,Aga
404,37,15
390,35,13
218,20,26
135,34,16
260,23,17


Pas mal comme modèle ;-), on ne se trompe pas vraiment souvent de qui gagne le match et en plus les scores restent assez proches.

# 3. Evaluation du modèle par validation croisée

In [18]:
#Validation croisée sur 5 folds
kf = KFold(n_splits=5,random_state=42)

In [19]:
# Validation croiséee
for train_index, test_index in kf.split(X):
    # Les index des folds
    X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index,:], y.iloc[test_index,:]
    
    #fit and predict
    clf2 = MultiOutputRF(n_estimators=2000).fit(X_train, y_train)
    Ypred2 = clf2.predict(X_test)
    
    #changer y_pred au bon format
    y_pred_df = pd.DataFrame(Ypred2.astype(int), columns=["For", 'Aga'])
    
    #eval
    print("Precision Loss/Win : ", get_loss_win_accuracy(y_test.copy(), y_pred_df.copy()))
    print("RMSE pour la norme 2 : ", norme2(y_test.copy(), y_pred_df.copy()))
    print("----------------------------------------------------------------------------------")

Precision Loss/Win :  0.8561643835616438
RMSE pour la norme 2 :  176.22712617528552
----------------------------------------------------------------------------------
Precision Loss/Win :  0.9041095890410958
RMSE pour la norme 2 :  168.56749390081114
----------------------------------------------------------------------------------
Precision Loss/Win :  0.8561643835616438
RMSE pour la norme 2 :  177.341478509682
----------------------------------------------------------------------------------
Precision Loss/Win :  0.8424657534246576
RMSE pour la norme 2 :  184.70787747142785
----------------------------------------------------------------------------------
Precision Loss/Win :  0.9103448275862069
RMSE pour la norme 2 :  205.9441671910132
----------------------------------------------------------------------------------


On obtient une moyenne d'environ 87% de bonnes prédictions des victoires. L'algorithme dépasse même la barre des 90% sur certains folds, les performances sont donc assez satisfaisantes sur 729 matchs au total.

# 3. Prédiction des matchs de la coupe du monde 2019

Les données sur la coupe du monde collectées manuellement sont disponibles dans la base **data challenge.csv**.

In [20]:
rcw = pd.read_csv('Dataset/data challenge.csv', sep=';', encoding='latin-1')

In [21]:
rcw.sample(5)

,Team,Result,For,Aga,Diff,HTf,HTa,Opposition,Ground,Match Date,TeamRk,OppRk
3,new zealand,won,23,13,10,17,3,south africa,International Stadium Yokohama,21/09/2019,89.40,87.34
17,scotland,won,34,0,34,20,0,samoa,Kobe Misaki Stadium,30/09/2019,80.54,70.80
30,scotland,won,61,0,61,21,0,russia,Shizuoka Stadium Ecopa,09/10/2019,80.54,63.09
1,australia,won,39,21,18,12,14,fiji,Sapporo Dome,21/09/2019,84.05,77.43
5,ireland,won,27,3,24,19,3,scotland,Hanazono Rugby Stadium,22/09/2019,89.47,81.00


In [22]:
dM2 = dataManager(dataframe=rcw)

In [23]:
dM2.data.head(5)

,Team,Result,For,Aga,Diff,HTf,HTa,Opposition,Ground,Match Date,TeamRk,OppRk
0,japan,won,30,10,20,12,7,russia,Tokyo Stadium,20/09/2019,76.70,64.81
1,australia,won,39,21,18,12,14,fiji,Sapporo Dome,21/09/2019,84.05,77.43
2,france,won,23,21,2,20,3,argentina,Tokyo Stadium,21/09/2019,79.72,76.29
3,new zealand,won,23,13,10,17,3,south africa,International Stadium Yokohama,21/09/2019,89.40,87.34
4,italy,won,47,22,25,21,7,namibia,Hanazono Rugby Stadium,22/09/2019,72.04,61.01


In [24]:
dM2.generate_date_features('Match Date')
dM2.data.head(5)

,Team,Result,For,Aga,Diff,HTf,HTa,Opposition,Ground,Match Date,TeamRk,OppRk,year,month,day0fYear,dayOfWeek
0,japan,won,30,10,20,12,7,russia,Tokyo Stadium,2019-09-20,76.70,64.81,2019,9,263,4
1,australia,won,39,21,18,12,14,fiji,Sapporo Dome,2019-09-21,84.05,77.43,2019,9,264,5
2,france,won,23,21,2,20,3,argentina,Tokyo Stadium,2019-09-21,79.72,76.29,2019,9,264,5
3,new zealand,won,23,13,10,17,3,south africa,International Stadium Yokohama,2019-09-21,89.40,87.34,2019,9,264,5
4,italy,won,47,22,25,21,7,namibia,Hanazono Rugby Stadium,2019-09-22,72.04,61.01,2019,9,265,6


In [25]:
#Faire la prédiction
fit_predict_new(data1=dM.data.copy(), data2=dM2.data.copy(), eval=True)

Precision Loss/Win :  0.8611111111111112
RMSE pour la norme 2 :  104.69479452198185
----------------------------------------------------------------------------------


,home,For,Aga,away
0,japan,32,14,russia
1,australia,33,18,fiji
2,france,25,16,argentina
3,new zealand,19,17,south africa
4,italy,52,19,namibia
5,ireland,26,11,scotland
6,england,45,8,tonga
7,wales,39,14,georgia
8,russia,14,25,samoa
9,fiji,33,15,uruguay


In [26]:
#On les compare aux résultats des vrais matchs
dM2.data[["Team", "For", "Aga", "Opposition"]]

,Team,For,Aga,Opposition
0,japan,30,10,russia
1,australia,39,21,fiji
2,france,23,21,argentina
3,new zealand,23,13,south africa
4,italy,47,22,namibia
5,ireland,27,3,scotland
6,england,35,3,tonga
7,wales,43,14,georgia
8,russia,9,34,samoa
9,fiji,27,30,uruguay


On constate que le RMSE a beaucoup diminué car là on est que sur 36 matchs prédits. Les prédictions sont assez convaicantes.

# 4. Prédiction des matchs de la phase finale et détermination du vainqueur de la compétition

Il est également possible de fournir un dataset dont les scores des colonnes sont vides afin de les remplir selon les prédictions. Nous avons créé un fichier csv que nous avons rempli au fur et à mesure que la l'agorithme prédisait un match de la compétition. Au départ, le fichier ressemble à ça :

In [27]:
new_matchs = pd.read_csv('Dataset/new_matchs.csv', sep=';')
new_matchs

,Team,Result,For,Aga,Diff,HTf,HTa,Opposition,Ground,Match Date,TeamRk,OppRk
0,united states of america,NaN,NaN,NaN,NaN,NaN,NaN,tonga,NaN,13/10/2019,72.18,69.22
1,wales,NaN,NaN,NaN,NaN,NaN,NaN,uruguay,NaN,13/10/2019,87.32,67.41
2,japan,NaN,NaN,NaN,NaN,NaN,NaN,scotland,NaN,13/10/2019,80.70,80.54
3,england,NaN,NaN,NaN,NaN,NaN,NaN,australia,NaN,19/10/2019,88.13,73.67
4,new zealand,NaN,NaN,NaN,NaN,NaN,NaN,ireland,NaN,19/10/2019,90.98,85.93
5,wales,NaN,NaN,NaN,NaN,NaN,NaN,france,NaN,19/10/2019,87.32,79.72
6,japan,NaN,NaN,NaN,NaN,NaN,NaN,south africa,NaN,20/10/2019,80.70,85.75
7,england,NaN,NaN,NaN,NaN,NaN,NaN,new zealand,NaN,26/10/2019,88.13,90.98
8,wales,NaN,NaN,NaN,NaN,NaN,NaN,south africa,NaN,27/10/2019,87.32,85.75
9,england,NaN,NaN,NaN,NaN,NaN,NaN,south africa,NaN,01/11/2019,88.13,87.32


Nous allons prédire tous les 11 matchs présents dans la table ! Nous allons utiliser les résultats des matchs déjà jouées dans les poules pour mettre à jour les performances du modèle.

In [28]:
#Le nouveau dataset enrichi, contenant les nouveaux scores des matchs déjà joués au moment du datachallenge
new_dataset = pd.concat([dM.data, dM2.data[dM.data.columns]], ignore_index=True, sort=False).copy()

In [29]:
new_dataset.head()

,Team,For,Aga,Opposition,TeamRk,OppRk,year,month,day0fYear,dayOfWeek
0,wales,16.0,21.0,england,81.64,84.85,2015,6,153,1
1,france,15.0,8.0,scotland,79.66,78.78,2015,7,183,3
2,scotland,8.0,15.0,france,78.78,79.66,2015,7,183,3
3,italy,3.0,26.0,ireland,71.19,85.48,2015,7,183,3
4,ireland,26.0,3.0,italy,85.48,71.19,2015,7,183,3


In [30]:
#Les matchs de la base et les matchs de poules déjà jouées
new_dataset.shape

(765, 10)

In [31]:
# Maintenant on va nettoyer la base contenant le sphases finales 'new_matchs'
full_cols = ["Team", "Opposition", "Match Date", "TeamRk", "OppRk"]
dM3 = dataManager(dataframe=new_matchs[full_cols].copy())
dM3.generate_date_features('Match Date')

'4 features successfully generated'

In [32]:
dM3

In [33]:
dM3.data.head()

,Team,Opposition,Match Date,TeamRk,OppRk,year,month,day0fYear,dayOfWeek
0,united states of america,tonga,2019-10-13,72.18,69.22,2019,10,286,6
1,wales,uruguay,2019-10-13,87.32,67.41,2019,10,286,6
2,japan,scotland,2019-10-13,80.70,80.54,2019,10,286,6
3,england,australia,2019-10-19,88.13,73.67,2019,10,292,5
4,new zealand,ireland,2019-10-19,90.98,85.93,2019,10,292,5


In [34]:
fit_predict_new(new_dataset.copy(), dM3.data.drop('Match Date', axis=1).copy())

Problème pour supprimer certaines colonnes :         Les noms d'une ou plusieurs colonnes sont mal spécifiées


,home,For,Aga,away
0,united states of america,22,17,tonga
1,wales,46,10,uruguay
2,japan,21,18,scotland
3,england,37,11,australia
4,new zealand,21,16,ireland
5,wales,23,13,france
6,japan,17,26,south africa
7,england,16,24,new zealand
8,wales,22,18,south africa
9,england,17,20,south africa


# 5. Rafinage du modèle en ajoutant les performances en attaque et defense des équipes.

Nous avons calculé les performances en attaque et defense des équipes en nous basant sur l'historique des matchs présents dans la base d'entrainement de départ.

In [35]:
base_att_def = pd.read_csv('Dataset/base_attaque_def.csv', sep=';')
base_att_def = base_att_def.iloc[:,1:]
base_att_def.sample(5, random_state=24)

,Team,year,Force_att,Force_deff
76,south africa,2017,"0,980336066","0,858763114"
81,tonga,2017,"1,676229796","0,879287599"
24,fiji,2019,"0,698635215","0,820305706"
69,scotland,2015,"1,051153552","0,916143025"
20,fiji,2015,"0,992663291","0,775536497"


Comme on le voit, les performances des équipes peuvent changer en fonction des années: le cas de **fiji**. Le détail des calculs de ces scores de performances sont détaillés dans un autre document.

In [36]:
# Exemple d'organisation des données
dM.data.merge(base_att_def, on=['Team', 'year'], how='left')\
    .merge(base_att_def, left_on=['Opposition', 'year'], right_on = ['Team', 'year'], how='left')\
    .drop('Team_y', axis=1).rename(columns ={'Team_x':'Team'})[['Team', 'Opposition', 'year', 'Force_att_x', 'Force_deff_x', 'Force_att_y', 'Force_deff_y']]\
    .head() #.sort_values(['Team', 'year'])

,Team,Opposition,year,Force_att_x,Force_deff_x,Force_att_y,Force_deff_y
0,wales,england,2015,"1,048039136","0,656704292","1,146074891","0,634814149"
1,france,scotland,2015,"0,92175877","0,94231854","1,051153552","0,916143025"
2,scotland,france,2015,"1,051153552","0,916143025","0,92175877","0,94231854"
3,italy,ireland,2015,"0,971177073","1,006946581","0,823459321","1,322529477"
4,ireland,italy,2015,"0,823459321","1,322529477","0,971177073","1,006946581"


* *force_att_x* désigne la force d'attaque de l'équipe à domicile pendant l'année et *force_att_y* celle de l'équipe extérieure.
* *force_def_x* désigne la force de défense de l'équipe à domicile pendant l'année et *force_def_y* celle de l'équipe extérieure.

In [37]:
# merging
df = dM.data.merge(base_att_def, on=['Team', 'year'], how='left')\
         .merge(base_att_def, left_on=['Opposition', 'year'], right_on = ['Team', 'year'], how='left')\
         .drop('Team_y', axis=1).rename(columns ={'Team_x':'Team'}).copy()
df2 = dM2.data.merge(base_att_def, on=['Team', 'year'], how='left')\
         .merge(base_att_def, left_on=['Opposition', 'year'], right_on = ['Team', 'year'], how='left')\
         .drop('Team_y', axis=1).rename(columns ={'Team_x':'Team'}).copy()

## Prédiction de la 2e base après entrainement sur la 1ere

In [38]:
fit_predict_new(df, df2, eval=True)

Precision Loss/Win :  0.8611111111111112
RMSE pour la norme 2 :  105.13324878457813
----------------------------------------------------------------------------------


,home,For,Aga,away
0,japan,31,13,russia
1,australia,34,18,fiji
2,france,24,17,argentina
3,new zealand,20,17,south africa
4,italy,53,18,namibia
5,ireland,26,10,scotland
6,england,42,8,tonga
7,wales,39,14,georgia
8,russia,15,19,samoa
9,fiji,32,18,uruguay


Le RMSE est plus faible que dans le cas 1 dans la section 3.

## Validation croisée

In [39]:
fit_predict_new(df, df2, cross_val=True)

Precision Loss/Win :  0.8627450980392157
RMSE pour la norme 2 :  179.11448852619378
----------------------------------------------------------------------------------
Precision Loss/Win :  0.8954248366013072
RMSE pour la norme 2 :  168.36567346107103
----------------------------------------------------------------------------------
Precision Loss/Win :  0.8888888888888888
RMSE pour la norme 2 :  185.88437266214714
----------------------------------------------------------------------------------
Precision Loss/Win :  0.8562091503267973
RMSE pour la norme 2 :  191.47584704082132
----------------------------------------------------------------------------------
Precision Loss/Win :  0.8758169934640523
RMSE pour la norme 2 :  221.7859328271295
----------------------------------------------------------------------------------


On remarque que la précision n'augmente pas significativement, malgré l'ajout de ces nouveaux features sur les performances en attaque et défense des équipes.